# Assistants API - Function Calling(함수 호출)

함수 호출을 사용하면 Assistants API에 함수를 설명하고 인수와 함께 호출해야 하는 함수를 지능적으로 반환하도록 할 수 있습니다.

이 예에서는 날씨 assistant를 만들고 assistant가 호출할 수 있는 도구로 `get_current_temp`, `get_rain_probability` 함수를 정의합니다. 사용자 쿼리에 따라 모델은 2023년 11월 6일 이후에 출시된 최신 모델을 사용하는 경우 병렬 함수 호출을 호출합니다. 병렬 함수 호출을 사용하는 예시에서는 assistant에게 오늘 특정 도시의 날씨가 어떨지와 비가 올 확률을 물어봅니다. 스트리밍으로 어시스턴트의 응답을 출력하는 방법도 보여줍니다.


In [1]:
import os
import openai
import sys
sys.path.append('./')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
from openai import OpenAI
client = OpenAI()

Model = "gpt-4o-mini"

### 1단계: 함수 정의
어시스턴트를 생성할 때 먼저 어시스턴트의 tools 매개변수 아래에 기능을 정의합니다.

In [3]:
#!pip install -q yfinance

import yfinance as yf

assistant = client.beta.assistants.create(
  instructions="당신은 주식 봇입니다. 제공된 function을 사용하여 질문에 답하세요.",
  model=Model,
  tools=[
    {
      "type": "function",
      "function": {
        "name": "yf_stock",
        "description": "특정 주식의 주가를 가져옵니다.",
        "parameters": {
          "type": "object",
          "properties": {
            "ticker": {
              "type": "string",
              "description": "주식 ticker"
            },
          },
          "required": ["ticker"]
        }
      }
    },
  ]
)

### 2단계: 스레드 생성 및 메시지 추가
사용자가 대화를 시작할 때 스레드를 생성하고 사용자가 질문을 하면 스레드에 메시지를 추가합니다.

In [4]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="오늘 Apple 주가는 어때요?",
)
message

Message(id='msg_T1whBv08d0sRHvING1lz1Kyh', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='오늘 Apple 주가는 어때요?'), type='text')], created_at=1730534843, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_xu7pIIv6oeh9Y39lEYNPZZ86')

### 3단계: 실행 시작
하나 이상의 기능을 트리거하는 사용자 메시지가 포함된 스레드에서 Run을 시작하면 Run이 `pending` 상태로 들어갑니다. 처리가 끝난 후 Run은 Run의 상태를 확인할 수 있는 `require_action` 상태로 전환됩니다. 이는 Run 실행을 계속하려면 도구(tools)를 실행하고 해당 출력을 Assistant에 제출해야 함을 나타냅니다. 우리 예의 경우에는 사용자 쿼리로 인해 병렬 함수 호출이 발생했음을 나타내는 두 개의 tool_call이 표시됩니다.

실행(Runs)은 비동기적으로 이루어지므로, 실행 객체(Run object)를 폴링하여 최종 상태에 도달할 때까지 상태를 모니터링해야 합니다. 편리함을 위해 'create_and_poll' SDK helper는 실행을 생성하고 완료될 때까지 폴링하는 작업을 도와줍니다. 실행이 완료되면 어시스턴트가 스레드에 추가한 메시지를 나열할 수 있습니다. 마지막으로, required_action에서 모든 도구 출력을 가져와 'submit_tool_outputs_and_poll' helper에 한 번에 제출하면 됩니다.

In [5]:
# 스레드 실행을 생성하고 결과를 폴링
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# 실행 상태가 완료되었는지 확인
if run.status == 'completed':
  # 실행된 스레드의 메시지를 가져옴
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  # 실행 상태를 출력
  print("not completed - ", run.status)

not completed -  requires_action


In [6]:
# 외부 함수 호출 필요 목록
run.required_action.submit_tool_outputs.tool_calls

[RequiredActionFunctionToolCall(id='call_Mx0Yuyytqm0uxdJN40ZZVnzR', function=Function(arguments='{"ticker":"AAPL"}', name='yf_stock'), type='function')]

In [7]:
# assistants 가 생성한 함수 호출에 필요한 parameter
run.required_action.submit_tool_outputs.tool_calls[0].function.arguments

'{"ticker":"AAPL"}'

In [8]:
# string을 dictionary로 변환
import json

argument_dict = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)
argument_dict

{'ticker': 'AAPL'}

In [9]:
# 도구 출력을 저장할 리스트를 정의
tool_outputs = []

# 필요한 작업 섹션의 각 도구를 순회
for tool in run.required_action.submit_tool_outputs.tool_calls:
  # 도구 이름에 따라 출력 값을 추가
  if tool.function.name == "yf_stock":
    df = yf.download(argument_dict['ticker'], progress=False)
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": str(df.Close.iloc[-1])
    })

tool_outputs

[{'tool_call_id': 'call_Mx0Yuyytqm0uxdJN40ZZVnzR',
  'output': '222.91000366210938'}]

In [10]:
# 도구 출력이 있는 경우 submit_tool_outputs_and_poll 이용
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    # 도구 출력이 성공적으로 제출되었음을 출력
    print("Tool outputs submitted successfully.")
  except Exception as e:
    # 도구 출력 제출 실패 메시지 출력
    print("Failed to submit tool outputs:", e)
else:
  # 제출할 도구 출력이 없음을 출력
  print("No tool outputs to submit.")

# 실행 상태가 완료되었는지 다시 확인
if run.status == 'completed':
  # 실행된 스레드의 메시지를 가져옴
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  # 실행 상태를 출력
  print(run.status)

Tool outputs submitted successfully.
SyncCursorPage[Message](data=[Message(id='msg_pfg8wF6aB2SVH3RkrMii8Ie7', assistant_id='asst_dGAR0IkzmiAZe4ESi6t1yTqv', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='오늘 Apple(AAPL) 주가는 $222.91입니다.'), type='text')], created_at=1730534862, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_SxvnRcxdld7R5irtRbpIT5Sb', status=None, thread_id='thread_xu7pIIv6oeh9Y39lEYNPZZ86'), Message(id='msg_T1whBv08d0sRHvING1lz1Kyh', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='오늘 Apple 주가는 어때요?'), type='text')], created_at=1730534843, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_xu7pIIv6oeh9Y39lEYNPZZ86')], object='list', first_id='msg_pfg8wF6aB2SVH3RkrMii8Ie7', last_id='msg_T1whBv08d0sRHvING1lz1Kyh',

In [11]:
print(messages.data[0].content[0].text.value)

오늘 Apple(AAPL) 주가는 $222.91입니다.


### Thread에 새로운 message 추가 및 Run 생성

In [12]:
message = client.beta.threads.messages.create(
      thread_id=thread.id,
      role="user",
      content="Apple의 주요 생산품은 무엇인가요??"
    )

In [13]:
# 스레드 실행을 생성하고 결과를 폴링
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

# 실행 상태가 완료되었는지 확인
if run.status == 'completed':
  # 실행된 스레드의 메시지를 가져옴
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  # 실행 상태를 출력
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_pH0kGROufI4rIMJYapcr0JPW', assistant_id='asst_dGAR0IkzmiAZe4ESi6t1yTqv', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Apple의 주요 생산품은 다음과 같습니다:\n\n1. **아이폰 (iPhone)**: 스마트폰 제품군으로, Apple의 가장 유명하고 수익성이 높은 제품입니다.\n2. **아이패드 (iPad)**: 태블릿 컴퓨터로, 다양한 크기와 성능으로 제공됩니다.\n3. **맥 (Mac)**: 개인용 컴퓨터 및 노트북 라인업, 다양한 모델이 존재합니다.\n4. **애플 워치 (Apple Watch)**: 스마트워치 제품으로, 건강 모니터링 및 연결 기능을 제공합니다.\n5. **에어팟 (AirPods)**: 무선 이어폰으로, 음악 청취 및 통화에 사용됩니다.\n6. **소프트웨어 및 서비스**: iOS, macOS, App Store, Apple Music, iCloud 등 다양한 소프트웨어 및 서비스도 중요한 수익원입니다.\n\nApple은 하드웨어, 소프트웨어 및 서비스를 통합하여 혁신적인 사용자 경험을 제공하는 데 중점을 두고 있습니다.'), type='text')], created_at=1730534877, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_G8Agdn1hjumrq2HtIwZl3OsB', status=None, thread_id='thread_xu7pIIv6oeh9Y39lEYNPZZ86'), Message(id='msg_Mz772Zw2rs5LIJHLa8WusJiq', assistant_id=None, attachments=[

In [14]:
print(messages.data[0].content[0].text.value)

Apple의 주요 생산품은 다음과 같습니다:

1. **아이폰 (iPhone)**: 스마트폰 제품군으로, Apple의 가장 유명하고 수익성이 높은 제품입니다.
2. **아이패드 (iPad)**: 태블릿 컴퓨터로, 다양한 크기와 성능으로 제공됩니다.
3. **맥 (Mac)**: 개인용 컴퓨터 및 노트북 라인업, 다양한 모델이 존재합니다.
4. **애플 워치 (Apple Watch)**: 스마트워치 제품으로, 건강 모니터링 및 연결 기능을 제공합니다.
5. **에어팟 (AirPods)**: 무선 이어폰으로, 음악 청취 및 통화에 사용됩니다.
6. **소프트웨어 및 서비스**: iOS, macOS, App Store, Apple Music, iCloud 등 다양한 소프트웨어 및 서비스도 중요한 수익원입니다.

Apple은 하드웨어, 소프트웨어 및 서비스를 통합하여 혁신적인 사용자 경험을 제공하는 데 중점을 두고 있습니다.


In [15]:
# messages 객체의 데이터에서 role과 content 값을 추출하여 리스트에 저장
values = [(messages.data[i].role, messages.data[i].content[0].text.value) for i in range(len(messages.data)-1, -1, -1)]

# 추출한 값들을 출력
for role, value in values:
    print(f"role: {role}")  # 역할(role) 출력
    print(value)  # 내용(value) 출력
    print(150*"-")  # 구분선 출력

role: user
오늘 Apple 주가는 어때요?
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
오늘 Apple(AAPL) 주가는 $222.91입니다.
------------------------------------------------------------------------------------------------------------------------------------------------------
role: user
Apple의 주요 생산품은 무엇인가요??
------------------------------------------------------------------------------------------------------------------------------------------------------
role: assistant
Apple의 주요 생산품은 다음과 같습니다:

1. **아이폰 (iPhone)**: 스마트폰 제품군으로, Apple의 가장 유명하고 수익성이 높은 제품입니다.
2. **아이패드 (iPad)**: 태블릿 컴퓨터로, 다양한 크기와 성능으로 제공됩니다.
3. **맥 (Mac)**: 개인용 컴퓨터 및 노트북 라인업, 다양한 모델이 존재합니다.
4. **애플 워치 (Apple Watch)**: 스마트워치 제품으로, 건강 모니터링 및 연결 기능을 제공합니다.
5. **에어팟 (AirPods)**: 무선 이어폰으로, 음악 청취 및 통화에 사용됩니다.
6. **소프트웨어 및 서비스**: iOS, macOS, App Store, Apple Music, iCloud 등 다양한 소프트웨어 및 서비스도 중요한 수익원입니다.

Apple은 하드웨어, 소프트웨어 및 서

## 실습 : Prompt 수정하여 실행